In [14]:
import streamlit as st
import pandas as pd
import time
import requests
from tqdm import tqdm
import random
from pprint import pprint
import pycountry

In [2]:
excel_file = "../data/4_GEOMAR-relevant_COG_grantees_and_panel_members_2013-2024.xlsx"
dashboard_file = "../data/List of funded projects(armUGTS).xlsx"

In [50]:
df_excel = pd.read_excel(excel_file, sheet_name="Grantees")  # Lade alle Tabellenblätter
df_excel["Name"] = df_excel["First Name"].astype(str) + " " + df_excel["Last Name"].astype(str)
df_excel["Name"] = df_excel["Name"].str.strip()
unique_panels = df_excel["Panel"].unique().tolist()
df_excel

,Last Name,First Name,Host Institution(s),Host Country,Acronym,Project Title,Abstract,Panel,Domain,Call,CORDIS Link,Name
0,Baudisch,Annette,"University of Southern Denmark [999904616,DK]",DK,Born Once - Die Once,Towards a transdisciplinary demographic theory...,Fertility and mortality fuel population renewa...,SH7,SH,ERC-2021-COG,https://cordis.europa.eu/project/id/101043983,Annette Baudisch
1,Tavoni,Massimo,"Polytechnic University of Milan [999879881,IT]",IT,EUNICE,Debiasing the uncertainties of climate stabili...,Mathematical models have become central tools ...,SH7,SH,ERC-2021-COG,https://cordis.europa.eu/project/id/101044703,Massimo Tavoni
2,Alkemade,Floortje,"Eindhoven University of Technology [999977269,NL]",NL,FAST,Fostering social tipping dynamics to Accelerat...,Staying within a maximum global warming of 1.5...,SH7,SH,ERC-2021-COG,https://cordis.europa.eu/project/id/101044076,Floortje Alkemade
3,Rufat,Samuel,"Polytechnic School - Palaiseau [999864264,FR]",FR,FiBeGa,Filling the Behavioral Gap in Disaster Risk Re...,Do only foolish people drown and only compulsi...,SH7,SH,ERC-2021-COG,https://cordis.europa.eu/project/id/101044374,Samuel Rufat
4,Toivonen,Tuuli,"University of Helsinki [999994535,FI]",FI,GREENTRAVEL,GREENER URBAN TRAVEL ENVIRONMENTS FOR EVERYONE...,Greenery is an integral part of healthy and li...,SH7,SH,ERC-2021-COG,https://cordis.europa.eu/project/id/101044906,Tuuli Toivonen
...,...,...,...,...,...,...,...,...,...,...,...,...
474,Hehenberger,Elisabeth,"Biology Centre (CAS) [999520108,CZ]",CZ,KLEPTOS,Using Kleptoplasty to illuminate the Evolution...,"Plastids, the photosynthetic organelles of euk...",LS8,LS,ERC-2024-COG,https://cordis.europa.eu/project/id/101170544,Elisabeth Hehenberger
475,Condamine,Fabien,National Centre for Scientific Research (CNRS)...,FR,ORION,ORigin and evolutION of biodiversity at the mi...,"Patterns of species distribution across space,...",LS8,LS,ERC-2024-COG,https://cordis.europa.eu/project/id/101170500,Fabien Condamine
476,Klein,Tamir,"Weizmann Institute [999979306,IL]",IL,RESILFOREST,Improving forest drought resilience through tr...,Trees play a major role in Earth’s water and c...,LS8,LS,ERC-2024-COG,https://cordis.europa.eu/project/id/101167854,Tamir Klein
477,Raina,Jean-Baptiste,National Centre for Scientific Research (CNRS)...,FR,SymbioCues,Characterising the role of microbial behaviour...,Coral reefs are the most diverse marine ecosys...,LS8,LS,ERC-2024-COG,https://cordis.europa.eu/project/id/101170326,Jean-Baptiste Raina


In [15]:
# Funktion, um den ISO-Alpha-2-Code zu bekommen
def get_country_code(name):
    try:
        return pycountry.countries.lookup(name).alpha_2
    except LookupError:
        return None  # Falls kein Land gefunden wird



In [80]:
df_dashboard = pd.read_excel(dashboard_file)
print(df_dashboard.columns.tolist())
selected_columns = ["Researcher(s)", "Host Institution(s)", "Country", "Acronym", "Project Title", "Abstract", "Panel", "Domain", "Call", "CORDIS Link"]



df_dashboard = df_dashboard[selected_columns]
# rename columns
df_dashboard.rename(columns={
    "Researcher(s)": "Name",
    "Country": "Host Country", 
}, inplace=True)
# Neue Spalte mit Länderkürzel
df_dashboard['Host Country'] = df_dashboard['Host Country'].apply(get_country_code)


# select from Life Sciences (LS) only the word in ()
df_dashboard["Domain"] = df_dashboard["Domain"].str.extract(r'\((.*?)\)')

# only select the word bevore - in LS1 - Molecules of Life: Biological Mechanisms...
df_dashboard["Panel"] = df_dashboard["Panel"].str.split('-').str[0].str.strip()

# only select the Panel values that are in the unique_panels list
df_dashboard = df_dashboard[df_dashboard["Panel"].isin(unique_panels)]

#df_dashboard.sort_values(by="Name", inplace=True)
df_dashboard

['Programme', 'Acronym', 'Project Title', 'Abstract', 'Researcher(s)', 'Host Institution(s)', 'Country', 'Region', 'Project Number', 'Call', 'Grant Type', 'Domain', 'Panel', 'Call Year', 'Start Date', 'End Date', 'EU contribution', 'CORDIS Link']


,Name,Host Institution(s),Host Country,Acronym,Project Title,Abstract,Panel,Domain,Call,CORDIS Link
922,Anton Potapov,Senckenberg Society for Nature Research [99783...,DE,CARBONWEB,Food web control of the global topsoil carbon ...,Soil food webs process 90% of the terrestrial ...,LS8,LS,ERC-2024-COG,https://cordis.europa.eu/project/id/101170898
923,Jussi Lehtonen,"University of Jyvaskyla [999842245,FI]",FI,CAUSEX,Unified theory of sexual selection,Sexual selection and sexual strategies include...,LS8,LS,ERC-2024-COG,https://cordis.europa.eu/project/id/101171163
924,Arnau Sebé-Pedrós,"Centre for Genomic Regulation [999544455,ES]",ES,CELL-SYSTEMATICS,Animal cell types across evolutionary timescal...,Cell types are the basic constituents of anima...,LS8,LS,ERC-2024-COG,https://cordis.europa.eu/project/id/101170846
925,Arnaud Sentis,French National Institute for Agricultural Res...,FR,ClimateCountDown,Using thermal limits to understand community r...,When will communities reach their upper therma...,LS8,LS,ERC-2024-COG,https://cordis.europa.eu/project/id/101169909
926,Henrik Salje,"University of Cambridge [999977172,UK]",GB,DENGEN,"Linking vector, virus and immunity to the evol...","Across pathogens, it remains unclear why some ...",LS8,LS,ERC-2024-COG,https://cordis.europa.eu/project/id/101170844
...,...,...,...,...,...,...,...,...,...,...
3829,David Stuckler,"Bocconi University Milan [999838850,IT]",IT,PRA,Pension reforms and healthy ageing in Italy an...,The COVID-19 pandemic has forced many European...,SH7,SH,ERC-2021-COG,https://cordis.europa.eu/project/id/101045534
3830,Daniela Cristina Antelmi Pigosso,"Technical University of Denmark [999990655,DK]",DK,REBOUNDLESS,Towards the prevention of rebound effects with...,Society´s most well-intended efforts to solve ...,SH7,SH,ERC-2021-COG,https://cordis.europa.eu/project/id/101043931
3831,Irene Perez Ibarra,"University of Zaragoza [999898214,ES]",ES,RESILIENT RULES,Evolution of institutional diversity in a chan...,Institutional diversity is considered as impor...,SH7,SH,ERC-2021-COG,https://cordis.europa.eu/project/id/101044225
3832,Sonja Haustein,"Technical University of Denmark [999990655,DK]",DK,URGENT,"Choice, necessity or chance? Understanding beh...",Despite various soft and hard measures as well...,SH7,SH,ERC-2021-COG,https://cordis.europa.eu/project/id/101044339


In [83]:
# merge both dataframes on Name
df_combined = pd.concat([df_excel, df_dashboard], ignore_index=True, sort=False)
# Maske erstellen
mask = df_combined["Last Name"].isnull() & df_combined["First Name"].isnull()

# Überprüfen, ob die Spalte "Name" korrekt formatiert ist
df_combined["Name"] = df_combined["Name"].str.strip()

# Spalte "Name" in "First Name" und "Last Name" aufteilen
split_names = df_combined.loc[mask, "Name"].str.split(' ', n=1, expand=True)

split_names.columns = ["First Name", "Last Name"]

# Fallback für fehlende Nachnamen
split_names["Last Name"] = split_names["Last Name"]

# Zuweisung der aufgeteilten Namen
df_combined.loc[mask, ["First Name", "Last Name"]] = split_names

df_combined = df_combined.drop_duplicates().reset_index(drop=True)
# Ergebnis anzeigen
df_combined

,Last Name,First Name,Host Institution(s),Host Country,Acronym,Project Title,Abstract,Panel,Domain,Call,CORDIS Link,Name
0,Baudisch,Annette,"University of Southern Denmark [999904616,DK]",DK,Born Once - Die Once,Towards a transdisciplinary demographic theory...,Fertility and mortality fuel population renewa...,SH7,SH,ERC-2021-COG,https://cordis.europa.eu/project/id/101043983,Annette Baudisch
1,Tavoni,Massimo,"Polytechnic University of Milan [999879881,IT]",IT,EUNICE,Debiasing the uncertainties of climate stabili...,Mathematical models have become central tools ...,SH7,SH,ERC-2021-COG,https://cordis.europa.eu/project/id/101044703,Massimo Tavoni
2,Alkemade,Floortje,"Eindhoven University of Technology [999977269,NL]",NL,FAST,Fostering social tipping dynamics to Accelerat...,Staying within a maximum global warming of 1.5...,SH7,SH,ERC-2021-COG,https://cordis.europa.eu/project/id/101044076,Floortje Alkemade
3,Rufat,Samuel,"Polytechnic School - Palaiseau [999864264,FR]",FR,FiBeGa,Filling the Behavioral Gap in Disaster Risk Re...,Do only foolish people drown and only compulsi...,SH7,SH,ERC-2021-COG,https://cordis.europa.eu/project/id/101044374,Samuel Rufat
4,Toivonen,Tuuli,"University of Helsinki [999994535,FI]",FI,GREENTRAVEL,GREENER URBAN TRAVEL ENVIRONMENTS FOR EVERYONE...,Greenery is an integral part of healthy and li...,SH7,SH,ERC-2021-COG,https://cordis.europa.eu/project/id/101044906,Tuuli Toivonen
...,...,...,...,...,...,...,...,...,...,...,...,...
633,Hulvej Rod,Naja,"University of Copenhagen [999991043,DK]",DK,LAYERS,The multiple LAYERS of childhood adversity,Health inequality is a major societal challeng...,SH7,SH,ERC-2023-COG,https://cordis.europa.eu/project/id/101124807,Naja Hulvej Rod
634,Dionisio Pérez-Blanco,Carlos,IMDEA Agua - Madrid Institute of Advanced Stud...,ES,WaterTheft,Tackling Water Theft: Forecasting Adaptation S...,Water theft claims between 30% and 50% of the ...,SH7,SH,ERC-2023-COG,https://cordis.europa.eu/project/id/101124884,Carlos Dionisio Pérez-Blanco
635,Cuni Sanchez,Aida,Norwegian University of Life Sciences [9999029...,NO,AFRIFOR,Improving tropical forest conservation in Africa,The importance of non-timber forest products (...,SH7,SH,ERC-2022-COG,https://cordis.europa.eu/project/id/101088599,Aida Cuni Sanchez
636,Jan Wouter Botzen,Willem,Free University and Medical Center Amsterdam (...,NL,INSUREADAPT,Public–Private Insurance Partnerships for Adap...,Natural disasters cause substantial economic l...,SH7,SH,ERC-2022-COG,https://cordis.europa.eu/project/id/101086783,Willem Jan Wouter Botzen
